# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [ ]:
%%file count_letters.py
from collections import Counter
def count_letters(file):
    with open(file, 'r', encoding='windows-1251') as fp:
        text = fp.read()
    return Counter(text)

In [ ]:
from count_letters import count_letters

In [ ]:
%%time
count_letters('data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
count_letters('data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [ ]:
import multiprocessing as mp

In [ ]:
mp.cpu_count()

In [ ]:
%%time
if __name__ == "__main__":

    files = ['data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', 
             'data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']

    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files) # один проц выполняет count_letter с параметром files
        # map - вернет управление, когда все процессы закончат работу и выдаст список результатов

In [ ]:
type(counters)

In [ ]:
counters

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [2]:
import csv

In [3]:
r_sum = 0
with open('data/recipes_full.csv', 'r') as f:
    incl_col = [1, 5, 6]                          # индексы нужных столбцов
    new_csv = []                              # новый список для нового файла
    reader = csv.reader(f, delimiter=",")
    for row in reader:
        col = list(row[i] for i in incl_col)
        new_csv.append(col)                   # заполняем новый список нужными столбцами
        
        r_sum += len(row[5][2:-2].split("', '"))


In [4]:
#найдем сколько строк должно быть в каждом файле с учетом количества тэгов
filesize = round(r_sum/8)
filesize

1767390

In [ ]:
# Разделяем на файлы
count = 0
name = 0
with open('data/recipes_full.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    head = ['id', 'tag', 'n_steps']

    for row in reader:
        if count >= filesize * name:
            # запись заголовков
            name += 1
            writer = csv.writer(open('id_tag_nsteps_' + str(name) + '.csv', 'w+'), delimiter=";")
            writer.writerow(head)

        for tag in row[5][2:-2].split("', '"):
            # запись id, каждого отдельного тега, n_steps
            writer.writerow([row[1], tag, row[6]])
            count += 1
    

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [5]:
import pandas as pd

In [6]:
%%time
def dict2(name):
    file = pd.read_csv(name, sep=';')
    r = file.groupby(['tag'])['n_steps'].mean()
    rr = r.to_dict()
    return rr
    

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [7]:
%%time
dict2("id_tag_nsteps_3.csv")
# dict2("id_tag_nsteps_4.csv")

CPU times: user 659 ms, sys: 91.6 ms, total: 750 ms
Wall time: 767 ms


{'1-day-or-more': 4.554892601431981,
 '15-minutes-or-less': 5.0599328338930505,
 '3-steps-or-less': 4.722672846658941,
 '30-minutes-or-less': 7.64619463161383,
 '4-hours-or-less': 10.026779422128259,
 '5-ingredients-or-less': 5.350758356648505,
 '60-minutes-or-less': 9.403606102635228,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.525116082735331,
 'a1-sauce': 3.506837229819144,
 'african': 4.394140625,
 'american': 7.55524861878453,
 'amish-mennonite': 3.6371951219512195,
 'angolan': 3.471104946833102,
 'appetizers': 6.2691420180406965,
 'apples': 4.887482900136799,
 'april-fools-day': 3.5055236411842685,
 'argentine': 3.564487632508834,
 'artichoke': 3.499778466991582,
 'asian': 6.449652777777778,
 'asparagus': 4.072821329088739,
 'australian': 4.271379703534778,
 'austrian': 3.563989408649603,
 'avocado': 3.6176858776946768,
 'bacon': 4.148622429181218,
 'baja': 3.599007668019847,
 'baked-beans': 3.4841129744042365,
 'baking': 3.67714528462192,
 'banana

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [8]:
import pandas as pd

In [12]:
%%time
def dict3():
    tg = []
    n_st = []
    new = pd.DataFrame()
    for i in range(8):
        file = pd.read_csv('id_tag_nsteps_' + str(i+1) + '.csv', sep=';')
        r = file.groupby(['tag'])['n_steps'].mean()
        rr = r.to_dict()
        
        for k, v in rr.items(): 
            tg.append(k)
            n_st.append(int(v))
            
    
    data = {'tag':tg, 'n_steps':n_st}
    new = pd.DataFrame(data)
    new = new.groupby(['tag'])['n_steps'].mean()
    new = new.to_dict()
    return new

dict3()

CPU times: user 4.79 s, sys: 608 ms, total: 5.39 s
Wall time: 5.45 s


{'1-day-or-more': 4.0,
 '15-minutes-or-less': 4.25,
 '3-steps-or-less': 4.0,
 '30-minutes-or-less': 7.0,
 '4-hours-or-less': 10.0,
 '5-ingredients-or-less': 5.0,
 '60-minutes-or-less': 9.0,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.0,
 'a1-sauce': 3.0,
 'african': 4.0,
 'american': 7.0,
 'amish-mennonite': 3.0,
 'angolan': 3.0,
 'appetizers': 6.0,
 'apples': 4.0,
 'april-fools-day': 3.0,
 'argentine': 3.0,
 'artichoke': 3.0,
 'asian': 6.0,
 'asparagus': 3.875,
 'australian': 4.0,
 'austrian': 3.0,
 'avocado': 3.0,
 'bacon': 4.0,
 'baja': 3.0,
 'baked-beans': 3.0,
 'baking': 3.0,
 'bananas': 3.625,
 'bar-cookies': 4.0,
 'barbecue': 4.0,
 'bass': 3.0,
 'bean-soup': 3.0,
 'beans': 5.0,
 'beans-side-dishes': 3.0,
 'bear': 3.0,
 'beef': 6.25,
 'beef-barley-soup': 3.0,
 'beef-crock-pot': 3.0,
 'beef-kidney': 3.0,
 'beef-liver': 3.0,
 'beef-organ-meats': 3.0,
 'beef-ribs': 3.0,
 'beef-sauces': 3.0,
 'beef-sausage': 3.0,
 'beginner-cook': 6.0,
 'beijing': 3.0,

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [13]:
import multiprocessing as mp

In [14]:
%%file dict3.py
import pandas as pd
def dict3(name):
    file = pd.read_csv(name, sep=';')
    r = file.groupby(['tag'])['n_steps'].mean()
    rr = r.to_dict()
    return rr

Overwriting dict3.py


In [15]:
from dict3 import dict3

In [16]:
%%time
if __name__ == "__main__":

    files = ["id_tag_nsteps_1.csv", 
             "id_tag_nsteps_2.csv",
            "id_tag_nsteps_3.csv",
            "id_tag_nsteps_4.csv",
            "id_tag_nsteps_5.csv",
            "id_tag_nsteps_6.csv",
            "id_tag_nsteps_7.csv",
            "id_tag_nsteps_8.csv"]

    with mp.Pool(processes=len(files)) as pool:
        res = pool.map(dict3, files) # один проц выполняет count_letter с параметром files
        # map - вернет управление, когда все процессы закончат работу и выдаст список результатов
        tg = []
        n_st = []
        count = 0
        for i in range(8):
            for k, v in res[i].items():
                tg.append(k)
                n_st.append(int(v))
            
        data = {'tag':tg, 'n_steps':n_st}
        new = pd.DataFrame(data)
        new = new.drop(labels = [0],axis = 0)
        new = new.groupby(['tag'])['n_steps'].mean()
        new = new.to_dict()
        print(new)

{'1-day-or-more': 4.0, '15-minutes-or-less': 4.25, '3-steps-or-less': 4.0, '30-minutes-or-less': 7.0, '4-hours-or-less': 10.0, '5-ingredients-or-less': 5.0, '60-minutes-or-less': 9.0, 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.0, 'a1-sauce': 3.0, 'african': 4.0, 'american': 7.0, 'amish-mennonite': 3.0, 'angolan': 3.0, 'appetizers': 6.0, 'apples': 4.0, 'april-fools-day': 3.0, 'argentine': 3.0, 'artichoke': 3.0, 'asian': 6.0, 'asparagus': 3.875, 'australian': 4.0, 'austrian': 3.0, 'avocado': 3.0, 'bacon': 4.0, 'baja': 3.0, 'baked-beans': 3.0, 'baking': 3.0, 'bananas': 3.625, 'bar-cookies': 4.0, 'barbecue': 4.0, 'bass': 3.0, 'bean-soup': 3.0, 'beans': 5.0, 'beans-side-dishes': 3.0, 'bear': 3.0, 'beef': 6.25, 'beef-barley-soup': 3.0, 'beef-crock-pot': 3.0, 'beef-kidney': 3.0, 'beef-liver': 3.0, 'beef-organ-meats': 3.0, 'beef-ribs': 3.0, 'beef-sauces': 3.0, 'beef-sausage': 3.0, 'beginner-cook': 6.0, 'beijing': 3.0, 'belgian': 3.0, 'berries': 4.5, 'beverages':

5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [17]:
import multiprocessing as mp

In [18]:
mp.cpu_count()

4

In [15]:
%%file dict4.py
from multiprocessing import Pool, Queue
import csv
import pandas as pd


def f(q_in, q_out):
    while True: 
        link = q_in.get()
        q_out.put(dict4(link))
        
def dict4(name):
    file = pd.read_csv(name, sep=';')
    r = file.groupby(['tag'])['n_steps'].mean()
    rr = r.to_dict()
    return rr



Overwriting dict4.py


In [1]:
%%time
import multiprocessing as mp
from dict4 import f


if __name__ == "__main__":
    
    files = ["id_tag_nsteps_1.csv", 
             "id_tag_nsteps_2.csv",
            "id_tag_nsteps_3.csv",
            "id_tag_nsteps_4.csv",
            "id_tag_nsteps_5.csv",
            "id_tag_nsteps_6.csv",
            "id_tag_nsteps_7.csv",
            "id_tag_nsteps_8.csv"]
    N = 2
    in_q = mp.Queue()
    out_q = mp.Queue()
    res = []

    for file in files:
        in_q.put(file)
        
    processes = []
    for i in range(2):
        process = mp.Process(target = f, args=(in_q, out_q))
        processes.append(process)
        
    for process in processes:
        process.start()

        
    for _ in files:
        result = out_q.get()
        res.append(result)
        
    
    tg = []
    n_st = []
    count = 0
    for i in range(8):
        for k, v in res[i].items():
            tg.append(k)
            n_st.append(int(v))

    data = {'tag':tg, 'n_steps':n_st}
    new = pd.DataFrame(data)
    new = new.drop(labels = [0],axis = 0)
    new = new.groupby(['tag'])['n_steps'].mean()
    new = new.to_dict()
    print(new)
        
print(res)        

[{'1-day-or-more': 4.4613270514330585, '15-minutes-or-less': 4.944268524382521, '3-steps-or-less': 4.693985544656686, '30-minutes-or-less': 7.589451799978116, '4-hours-or-less': 10.020288817281298, '5-ingredients-or-less': 5.3101368250698835, '60-minutes-or-less': 9.413485776017286, 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.5100087032201914, 'a1-sauce': 3.563061137238136, 'african': 4.4886929858183215, 'american': 7.762587635436584, 'amish-mennonite': 3.4982817869415808, 'angolan': 3.5143741707209197, 'appetizers': 6.221261337270618, 'apples': 4.8714744734023565, 'april-fools-day': 3.4729041222788326, 'argentine': 3.592967032967033, 'artichoke': 3.4732547597461467, 'asian': 6.379743522638053, 'asparagus': 4.009437833401724, 'australian': 4.1521655806822535, 'austrian': 3.6343574210298573, 'avocado': 3.5505569837189372, 'bacon': 4.1232208214721435, 'baja': 3.5275891341256367, 'baked-beans': 3.5220713073005094, 'baking': 3.635058721183123, 'bananas': 3.97